In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score

In [2]:
# Import the data
batchsize = 8
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

In [3]:
train_data = train_datagen.flow_from_directory(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train',
    target_size=(224, 224),  # DenseNet169 model input shape
    batch_size=batchsize,
    class_mode='categorical',
    subset='training'
)

validation_data = train_datagen.flow_from_directory(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train',
    target_size=(224, 224),  # DenseNet169 model input shape
    batch_size=batchsize,
    class_mode='categorical',
    subset='validation'
)

Found 64105 images belonging to 2 classes.
Found 16025 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\test',
    target_size=(224, 224),  # DenseNet169 model input shape
    batch_size=batchsize,
    class_mode='categorical'
)


Found 4768 images belonging to 2 classes.


In [5]:
# Model training
bmodel = DenseNet169(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False




51877672/51877672 [==============================] - 276s 5us/step


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

In [7]:
checkpoint = ModelCheckpoint(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=3
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=3,
    restore_best_weights=True
)

learning_rate = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=3,
)

callbacks = [checkpoint, earlystop, learning_rate]

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_data,
    steps_per_epoch=train_data.samples // batchsize,
    validation_data=validation_data,
    validation_steps=validation_data.samples // batchsize,
    callbacks=callbacks,
    epochs=5
)


Epoch 1/5


8013/8013 [==============================] - ETA: 0s - loss: 0.1958 - accuracy: 0.9224

KeyboardInterrupt: 

In [9]:
# Evaluate the model
acc_tr, loss_tr = model.evaluate(train_data)
print("Training Accuracy:", acc_tr)
print("Training Loss:", loss_tr)


8014/8014 [==============================] - 6488s 810ms/step - loss: 0.0909 - accuracy: 0.9667
Training Accuracy: 0.09093679487705231
Training Loss: 0.966695249080658


In [10]:
acc_vr, loss_vr = model.evaluate(validation_data)
print("Validation Accuracy:", acc_vr)
print("Validation Loss:", loss_vr)

2004/2004 [==============================] - 1608s 802ms/step - loss: 0.1409 - accuracy: 0.9534
Validation Accuracy: 0.14094875752925873
Validation Loss: 0.9533853530883789


In [11]:
acc_test, loss_test = model.evaluate(test_data)
print("Test Accuracy:", acc_test)
print("Test Loss:", loss_test)

596/596 [==============================] - 456s 764ms/step - loss: 0.2034 - accuracy: 0.9262
Test Accuracy: 0.20337563753128052
Test Loss: 0.926174521446228
